In [1]:
#load package
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

In [2]:
# path of the file
file_path= 'D:/Liuuu/Graduate@UCSD/Intern/DS/CapitalOne/transactions/transactions.txt' 
#create import function
def import_data(path):
    dfList = []
    df=pd.DataFrame()
    for iot_chunk in pd.read_json(file_path, lines=True,chunksize=100000) :
        dfList.append(iot_chunk)
    df = pd.concat(dfList,sort=False)
    return df

# pandas setting
pd.set_option('display.max_columns', None) 
transactions=import_data(file_path)
transactions.drop(['echoBuffer','merchantCity','merchantState','merchantZip','posOnPremises','recurringAuthInd'], axis=1,inplace=True)

# replace the "" to NaN 
transactions.replace("", np.nan, inplace=True)

# create 'Date' field
transactions['Date']=transactions['transactionDateTime'].apply(lambda x: x.split('T')[0])

# sort data by time
transactions['transactionDateTime'] = pd.to_datetime(transactions['transactionDateTime'])
transactions_sorted = transactions.groupby('accountNumber').apply(lambda x: x.sort_values(by='transactionDateTime', ascending=True))
transactions.head(10)

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,merchantCategoryCode,currentExpDate,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud,Date
0,737265056,737265056,5000,5000.00,2016-08-13 14:27:32,98.55,Uber,US,US,02,01,rideshare,06/2023,2015-03-14,2015-03-14,414,414,1803,PURCHASE,0.00,False,False,False,2016-08-13
1,737265056,737265056,5000,5000.00,2016-10-11 05:05:54,74.51,AMC #191138,US,US,09,01,entertainment,02/2024,2015-03-14,2015-03-14,486,486,767,PURCHASE,0.00,True,False,False,2016-10-11
2,737265056,737265056,5000,5000.00,2016-11-08 09:18:39,7.47,Play Store,US,US,09,01,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,0.00,False,False,False,2016-11-08
3,737265056,737265056,5000,5000.00,2016-12-10 02:14:50,7.47,Play Store,US,US,09,01,mobileapps,08/2025,2015-03-14,2015-03-14,486,486,767,PURCHASE,0.00,False,False,False,2016-12-10
4,830329091,830329091,5000,5000.00,2016-03-24 21:04:46,71.18,Tim Hortons #947751,US,US,02,01,fastfood,10/2029,2015-08-06,2015-08-06,885,885,3143,PURCHASE,0.00,True,False,False,2016-03-24
5,830329091,830329091,5000,5000.00,2016-04-19 16:24:27,30.76,In-N-Out #422833,US,US,02,01,fastfood,01/2020,2015-08-06,2015-08-06,885,885,3143,PURCHASE,0.00,True,False,False,2016-04-19
6,830329091,830329091,5000,5000.00,2016-05-21 14:50:35,57.28,Krispy Kreme #685312,US,US,02,01,fastfood,05/2020,2015-08-06,2015-08-06,885,885,3143,PURCHASE,0.00,True,False,False,2016-05-21
7,830329091,830329091,5000,5000.00,2016-06-03 00:31:21,9.37,Shake Shack #968081,US,US,05,01,fastfood,01/2021,2015-08-06,2015-08-06,885,885,3143,PURCHASE,0.00,True,False,False,2016-06-03
8,830329091,830329091,5000,4990.63,2016-06-10 01:21:46,523.67,Burger King #486122,NaN,US,02,01,fastfood,08/2032,2015-08-06,2015-08-06,885,885,3143,PURCHASE,9.37,True,False,False,2016-06-10
9,830329091,830329091,5000,5000.00,2016-07-11 10:47:16,164.37,Five Guys #510989,US,US,05,08,fastfood,04/2020,2015-08-06,2015-08-06,885,885,3143,PURCHASE,0.00,True,False,False,2016-07-11


In [3]:
transactions.shape

(786363, 24)

In [4]:
# save processed data into csv file
file_name = 'D:/Liuuu/Graduate@UCSD/Intern/DS/CapitalOne/transactions/processed_transactions.csv'
# Write to CSV file
transactions.to_csv(file_name, index=False)

# save processed data into PostgreSQL database
# PostgreSQL connection parameters
db_params = {
    'database': 'capitalone',
    'user': 'postgres',
    'password': 'Jiayou123!',
    'host': 'localhost',
    'port': '5432'
}

# create table
create_table_query = """
    CREATE TABLE IF NOT EXISTS card_transactions (
        accountNumber INT PRIMARY KEY,
        customerId INT,
        creditLimit DECIMAL,
        availableMoney DECIMAL,
        transactionDate DATE,
        transactionAmount DECIMAL,
        merchantName VARCHAR(255),
        merchantCountryCode VARCHAR(10),
        merchantCategoryCode VARCHAR(255),
        transactionType VARCHAR(50),
        currentBalance DECIMAL,
        cardPresent BOOLEAN
    );
"""

try:
    # Create SQLAlchemy engine
    engine = create_engine(f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["database"]}')

    # Connect to PostgreSQL database
    connection = engine.connect()

    # Execute the create table query
    connection.execute(create_table_query)

    print("create table schema successfully")

except Exception as e:
    print(f"error: {e}")

try:
    # Load DataFrame into PostgreSQL table
    transactions.to_sql('card_transactions', engine, if_exists='replace', index=False)

    print("Data loaded successfully")
    
except Exception as e2:
    print(f"error: {e2}")
    
        

create table schema successfully
Data loaded successfully


In [9]:
try:
    # execute the query
    sql_query = '''
        select "Date", "transactionType", sum("transactionAmount") as transactionAmount
        from card_transactions
        group by "Date", "transactionType";
    '''
    engine.execute(sql_query)
    results = pd.read_sql_query(sql_query, engine)

    
    name = 'sql_result.csv'
    path = 'D:/Liuuu/Graduate@UCSD/Intern/DS/CapitalOne/transactions/'
    output_csv_path = path + name
    results.to_csv(output_csv_path, index=False)

    print(f"Query results saved to {output_csv_path}")
except Exception as e3:
    print(f"error:{e3}")

Query results saved to D:/Liuuu/Graduate@UCSD/Intern/DS/CapitalOne/transactions/sql_result.csv


In [13]:
try:
    sql_query = '''
    SELECT "transactionType", count("transactionType")
    FROM card_transactions
    GROUP BY "transactionType";
    '''
    engine.execute(sql_query)
    results = pd.read_sql_query(sql_query, engine)

    print(results)
    for row in results:
        print(row)
except Exception as e4:
    print(f"error:{e4}")
finally:
    # Close the connection
    if connection:
        connection.close()

        transactionType   count
0  ADDRESS_VERIFICATION   20169
1              PURCHASE  745193
2              REVERSAL   20303
3                  None       0
transactionType
count
